In [1]:
# import json
import ast
from functools import partial
from typing import List
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.max_colwidth = 150

# Импорт данных

In [3]:
dataset = pd.read_parquet("train_pairs.parquet")
etl = pd.read_parquet("train_data.parquet")

## Транзитивность

In [4]:
dataset_trf = dataset.merge(dataset, how='inner', left_on='variantid2', right_on='variantid1')

In [5]:
# не совпадающие метки после соединения
dataset_new = dataset_trf[dataset_trf['target_x'] != dataset_trf['target_y']]
dataset_new_rtm = dataset_new[['target_y', 'variantid1_x', 'variantid2_y']]
dataset_new_rtm.columns = ['target', 'variantid1', 'variantid2']

dataset_new.head()

,target_x,variantid1_x,variantid2_x,target_y,variantid1_y,variantid2_y
14,1.0,89540591,732942986,0.0,732942986,797685052
31,1.0,91603949,485333946,0.0,485333946,523588747
37,0.0,91606616,91606667,1.0,91606667,607074418
38,1.0,96009344,229747040,0.0,229747040,418054843
50,1.0,147125215,158698192,0.0,158698192,452757806


In [7]:
# совпадающие метки после соединения
dataset_new2 = dataset_trf[dataset_trf['target_x'] == dataset_trf['target_y']]
dataset_new2_rtm = dataset_new2[['target_x', 'variantid1_x', 'variantid2_y']]
dataset_new2_rtm.columns = ['target', 'variantid1', 'variantid2']
dataset_new.head()

In [8]:
# новых данных:
print(((len(dataset_new_rtm) + len(dataset_new2_rtm) + len(dataset)) / len(dataset)) - 1)
print((len(dataset_new_rtm) + len(dataset_new2_rtm) + len(dataset)) - len(dataset))

0.3498727735368956
107250


In [9]:
# Объединение исходного и новых датафреймов.
dataset = pd.concat([dataset, dataset_new_rtm, dataset_new2_rtm], axis=0)

In [10]:
dataset.shape

(413790, 3)

In [11]:
# Пары + признаки
features = (
    dataset
    .merge(
        etl
        .add_suffix('1'),
        on="variantid1"
    )
    .merge(
        etl
        .add_suffix('2'),
        on="variantid2"
    )
)

In [13]:
df = features[['target', 'categories1', 'categories2','characteristic_attributes_mapping1', 'characteristic_attributes_mapping2']]

# Categories
* сравниваю словари категорий в разрезе таргета, кажется что в парах с целевой меткой доля одинаковых словарей должна быть больше

In [27]:
def compare_dicts(row):
    """
    Функция псравнивает 2 строки-словаря друг с другом.
    """
    return row['categories1'] == row['categories2']


df['categories1'] = df['categories1'].str.lower()
df['categories2'] = df['categories2'].str.lower()
df['comparison_result'] = df.apply(compare_dicts, axis=1) * 1

In [28]:
df.groupby('target', as_index=False).agg({'comparison_result':np.mean})

,target,comparison_result
0,0.0,0.977674
1,1.0,0.957258


* как то странно, казалось что в парах с таргетом совпадений должно быть больше
* посмотрим на такие странные пары

In [29]:
cat_missmatch = df[(df['target'] == 1) & (df['comparison_result'] == 0)]

In [41]:
def miss_match_info(cat_missmatch):
    """
    Функция берет на вход дф со строками словарями категорий.
    Возвращает новый дф с тремя столбцами:
    missmatch_cat - 1й уровень категории где есть различие в паре. Если расхождение было в категориях 2,3,4 то вернется 2.
    miss_match_obj - словарь, где ключ - категория с мисметчем по паре, значение - кортеж с названиями категорий
    missmatch_cat_cnt - в скольких уровнях был мисметч. Если расхождение было в категориях 2,3,4 то вернется 3.
    """

    miss_match_series = pd.Series(index=cat_missmatch.index, dtype='object')
    miss_match_category = pd.Series(index=cat_missmatch.index, dtype='int8')
    miss_match_count_category = pd.Series(index=cat_missmatch.index, dtype='int8')

    for i in cat_missmatch.index:
        cat1_dict = ast.literal_eval(cat_missmatch.loc[i,'categories1'])
        cat2_dict = ast.literal_eval(cat_missmatch.loc[i,'categories2'])
        if len(cat1_dict) == len(cat2_dict):
            miss_match_dict = {}
            cat_list = []
            for key in cat1_dict.keys():

                if cat1_dict[key] != cat2_dict[key]:
                    miss_match_dict[key] = tuple((cat1_dict[key], cat2_dict[key]))
                    miss_match_series.loc[i] = str(miss_match_dict)
                    cat_list.append(int(key))
            miss_match_category.loc[i] = min(cat_list)
            miss_match_count_category.loc[i] = len(cat_list)
            
    return pd.concat([miss_match_category, miss_match_series, miss_match_count_category], ignore_index=False, axis=1)

In [31]:
cat_missmatch = pd.concat([cat_missmatch, miss_match_info(cat_missmatch)], ignore_index=False, axis=1)

In [32]:
cat_missmatch.columns = ['target', 'categories1', 'categories2', 'characteristic_attributes_mapping1',
       'characteristic_attributes_mapping2','comparison_result', 'missmatch_cat', 'miss_match_obj', 'missmatch_cat_cnt']

In [42]:
# cat_missmatch.head()

In [33]:
cat_missmatch['missmatch_cat'].value_counts(normalize=True)

4.0    0.636035
3.0    0.336710
2.0    0.027255
Name: missmatch_cat, dtype: float64

In [34]:
cat_missmatch['missmatch_cat_cnt'].value_counts(normalize=True)

1.0    0.636035
2.0    0.336710
3.0    0.027255
Name: missmatch_cat_cnt, dtype: float64

* 2,7 % пар с меткой 1, имеют расхождение со 2-4 категории. Скорее всего это ошибки разметки. Предлагаю сносить.

In [35]:
# cat_missmatch[cat_missmatch['missmatch_cat_cnt'] == 3].to_excel('cat_missmatch.xlsx', index=False)

In [36]:
cat_missmatch_0 = df[(df['target'] == 0) & (df['comparison_result'] == 0)]

cat_missmatch_0 = pd.concat([cat_missmatch_0, miss_match_info(cat_missmatch_0)], ignore_index=False, axis=1,)

cat_missmatch_0.columns = ['target', 'categories1', 'categories2', 'characteristic_attributes_mapping1',
       'characteristic_attributes_mapping2','comparison_result', 'missmatch_cat', 'miss_match_obj', 'missmatch_cat_cnt']

In [37]:
cat_missmatch_0['missmatch_cat'].value_counts(normalize=True)

3.0    0.548604
4.0    0.446709
2.0    0.004687
Name: missmatch_cat, dtype: float64

In [38]:
cat_missmatch_0['missmatch_cat_cnt'].value_counts(normalize=True)

2.0    0.548604
1.0    0.446709
3.0    0.004687
Name: missmatch_cat_cnt, dtype: float64

In [39]:
cat_missmatch_0.head()

,target,categories1,categories2,characteristic_attributes_mapping1,characteristic_attributes_mapping2,comparison_result,missmatch_cat,miss_match_obj,missmatch_cat_cnt
125,0.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""зарядные устройства и док-станции"", ""4"": ""зарядное устройство""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""кабели и переходники"", ""4"": ""usb-концентратор""}","{""Страна-изготовитель"":[""Китай""],""Тип"":[""Беспроводное зарядное устройство""],""Бренд"":[""WIWU""],""Цвет товара"":[""серебристый""]}","{""Цвет товара"":[""серый""],""Бренд"":[""WIWU""],""Тип"":[""USB-концентратор""],""Электробезопасность"":[""Защита от высокого напряжения"",""Защита от короткого з...",0,3.0,"{'3': ('зарядные устройства и док-станции', 'кабели и переходники'), '4': ('зарядное устройство', 'usb-концентратор')}",2.0
508,0.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеорегистратор"", ""4"": ""видеорегистратор""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеонаблюдение"", ""4"": ""регистратор""}","{""Тип"":[""Регистратор""],""Гарантийный срок"":[""1 год""],""Бренд"":[""Hikvision""],""Вес товара, г"":[""2600""],""Страна-изготовитель"":[""Китай""],""Размеры, мм"":[...","{""Цвет товара"":[""белый""],""Тип"":[""Регистратор""],""Бренд"":[""Hikvision""],""Тип матрицы"":[""CMOS 1/2.8\""""],""Вид камеры"":[""IP камера""]}",0,3.0,"{'3': ('видеорегистратор', 'видеонаблюдение'), '4': ('видеорегистратор', 'регистратор')}",2.0
510,0.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеонаблюдение"", ""4"": ""регистратор""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеорегистратор"", ""4"": ""видеорегистратор""}","{""Общее количество пикселей"":[""12 Мпикс""],""Суммарный объем всех дисков, ГБ"":[""20000""],""Число подключаемых жестких дисков, max"":[""2""],""Сетевые прот...","{""Наличие дисплея"":[""Без дисплея""],""Диапазон рабочей температуры"":[""от -10 до +55""],""Тип"":[""Видеорегистратор""],""Комплектация"":[""Видеорегистратор H...",0,3.0,"{'3': ('видеонаблюдение', 'видеорегистратор'), '4': ('регистратор', 'видеорегистратор')}",2.0
793,0.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеонаблюдение"", ""4"": ""регистратор""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеонаблюдение"", ""4"": ""система видеонаблюдения""}","{""Гарантийный срок"":[""5 лет""],""Бренд"":[""Dahua""],""Тип"":[""Регистратор""]}","{""Страна-изготовитель"":[""Китай""],""Тип"":[""Система видеонаблюдения""],""Гарантийный срок"":[""2 года""],""Бренд"":[""Dahua""]}",0,4.0,"{'4': ('регистратор', 'система видеонаблюдения')}",1.0
796,0.0,"{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеонаблюдение"", ""4"": ""система видеонаблюдения""}","{""1"": ""epg"", ""2"": ""электроника"", ""3"": ""видеонаблюдение"", ""4"": ""регистратор""}","{""Количество в упаковке, шт"":[""1""],""Общее количество пикселей"":[""12 Мпикс""],""Материал купола"":[""Пластик""],""Бренд"":[""Dahua""],""Скорость съемки в мак...","{""Страна-изготовитель"":[""Китай""],""Макс. температура эксплуатации, С°"":[""55""],""Суммарный объем всех дисков, ГБ"":[""12288""],""Форматы файлов видео"":[""...",0,4.0,"{'4': ('система видеонаблюдения', 'регистратор')}",1.0


## Парсинг категорий.

In [242]:
features.reset_index(drop=True, inplace=True)

In [243]:
features.columns

Index(['target', 'variantid1', 'variantid2', 'name1', 'categories1',
       'color_parsed1', 'pic_embeddings_resnet_v11',
       'main_pic_embeddings_resnet_v11', 'name_bert_641',
       'characteristic_attributes_mapping1', 'name2', 'categories2',
       'color_parsed2', 'pic_embeddings_resnet_v12',
       'main_pic_embeddings_resnet_v12', 'name_bert_642',
       'characteristic_attributes_mapping2'],
      dtype='object')

In [244]:
nans = [np.nan for i in range(len(features))]
df_cat1 = pd.DataFrame({1:nans, 2:nans, 3:nans, 4:nans})
df_cat2 = pd.DataFrame({1:nans, 2:nans, 3:nans, 4:nans})

In [245]:
for i in tqdm_notebook(range(len(features))):    
    cat1 = ast.literal_eval(features.loc[i,'categories1'])
    cat2 = ast.literal_eval(features.loc[i,'categories2'])

    for j in range(1,5):  
        df_cat1.loc[i,j] = cat1[str(j)]
        df_cat2.loc[i,j] = cat2[str(j)] 

  0%|          | 0/413790 [00:00<?, ?it/s]

In [246]:
df_cat1.columns = ['cat1_1', 'cat2_1', 'cat3_1', 'cat4_1']
df_cat2.columns = ['cat1_2', 'cat2_2', 'cat3_2', 'cat4_2']

In [247]:
df_cat = pd.concat([df_cat1, df_cat2], axis= 1)

In [248]:
print(f'длина исходного дф {len(df_cat)}')
print(f"совпадений по категории 1го уровня - {len(df_cat[df_cat['cat1_1'] == df_cat['cat1_2']])}")
print(f"совпадений по категории 2го уровня - {len(df_cat[df_cat['cat2_1'] == df_cat['cat2_2']])}")
print(f"совпадений по категории 3го уровня - {len(df_cat[df_cat['cat3_1'] == df_cat['cat3_2']])}")
print(f"совпадений по категории 4го уровня - {len(df_cat[df_cat['cat4_1'] == df_cat['cat4_2']])}")

длина исходного дф 413790
совпадений по категории 1го уровня - 413790
совпадений по категории 2го уровня - 413541
совпадений по категории 3го уровня - 408057
совпадений по категории 4го уровня - 400591


* категория 1го уровня совпадает на всех парах независимо от таргета, ее можно исключить/

In [249]:
df_cat.isnull().sum()

cat1_1    0
cat2_1    0
cat3_1    0
cat4_1    0
cat1_2    0
cat2_2    0
cat3_2    0
cat4_2    0
dtype: int64

In [250]:
features = pd.concat([features, df_cat], axis=1)

In [251]:
features.isnull().any()

target                                False
variantid1                            False
variantid2                            False
name1                                 False
categories1                           False
color_parsed1                          True
pic_embeddings_resnet_v11              True
main_pic_embeddings_resnet_v11        False
name_bert_641                         False
characteristic_attributes_mapping1     True
name2                                 False
categories2                           False
color_parsed2                          True
pic_embeddings_resnet_v12              True
main_pic_embeddings_resnet_v12        False
name_bert_642                         False
characteristic_attributes_mapping2     True
cat1_1                                False
cat2_1                                False
cat3_1                                False
cat4_1                                False
cat1_2                                False
cat2_2                          

## Работа с null

In [252]:
# считаем null в разрезе категории 2-го уровня для 1го товара
cat2_nulls_1 = features.groupby('cat2_1', as_index=False).agg({'characteristic_attributes_mapping1':['count', np.size]})
cat2_nulls_1.columns = ['_'.join(col).strip() for col in cat2_nulls_1.columns.values]
cat2_nulls_1.columns = ['cat2_1', 'count','size']

cat2_nulls_1['nulls_by_category'] =  cat2_nulls_1['size'] - cat2_nulls_1['count']

cat2_nulls_1

,cat2_1,count,size,nulls_by_category
0,Детские товары,3,3,0
1,Строительство и ремонт,3,3,0
2,Электроника,413767,413784,17


In [253]:
# считаем null в разрезе категории 2-го уровня для 2го товара
cat2_nulls_2 = features.groupby('cat2_2', as_index=False).agg({'characteristic_attributes_mapping2':['count', np.size]})
cat2_nulls_2.columns = ['_'.join(col).strip() for col in cat2_nulls_2.columns.values]
cat2_nulls_2.columns = ['cat2_2', 'count','size']

cat2_nulls_2['nulls_by_category'] =  cat2_nulls_2['size'] - cat2_nulls_2['count']

cat2_nulls_2

,cat2_2,count,size,nulls_by_category
0,Автотовары,34,34,0
1,Бытовая техника,5,5,0
2,Бытовая химия,3,3,0
3,Галантерея и украшения,2,2,0
4,Детские товары,23,23,0
5,Дом и сад,19,19,0
6,Канцелярские товары,24,24,0
7,Спорт и отдых,13,13,0
8,Строительство и ремонт,53,53,0
9,Товары для взрослых,1,1,0


* Пропуски в характеристика встречаются только в категории Электроника.
* Предлагаю удалить.

In [254]:
features.isnull().sum()

target                                     0
variantid1                                 0
variantid2                                 0
name1                                      0
categories1                                0
color_parsed1                          50118
pic_embeddings_resnet_v11             104326
main_pic_embeddings_resnet_v11             0
name_bert_641                              0
characteristic_attributes_mapping1        17
name2                                      0
categories2                                0
color_parsed2                          56488
pic_embeddings_resnet_v12             108936
main_pic_embeddings_resnet_v12             0
name_bert_642                              0
characteristic_attributes_mapping2        14
cat1_1                                     0
cat2_1                                     0
cat3_1                                     0
cat4_1                                     0
cat1_2                                     0
cat2_2    

* много пропусков по цвету, попробую заполнить топ цветом в разрезе категория-2 уровня

In [293]:
etl[etl['color_parsed'].notnull()]['color_parsed'].apply(lambda x: len(list(x))).value_counts(normalize=True)

1     0.745444
2     0.203408
3     0.033928
4     0.010994
5     0.002535
6     0.002506
8     0.000401
7     0.000251
9     0.000188
15    0.000119
16    0.000058
12    0.000050
10    0.000050
14    0.000024
11    0.000021
13    0.000013
18    0.000005
17    0.000003
Name: color_parsed, dtype: float64

In [295]:
etl.loc[etl['color_parsed'].notnull(), 'color_parsed'] = etl[etl['color_parsed'].notnull()]['color_parsed'].apply(lambda x: list(x))

In [296]:
etl['len_color_parsed'] = np.nan
etl.loc[etl['color_parsed'].notnull(), 'len_color_parsed'] = etl[etl['color_parsed'].notnull()]['color_parsed'].apply(lambda x: len(x))

In [299]:
print(etl[etl['len_color_parsed'] == 2].shape)
print((etl[etl['len_color_parsed'] == 2].shape)

(77021, 9)


In [255]:
# # перевожу поле с цветом к формату строки, там где список из нескольких цветов беру первый вариант.

# features.loc[features['color_parsed1'].notnull(), 'color_parsed1'] = features.loc[features['color_parsed1'].notnull(), 'color_parsed1'].apply(lambda x: list(x)[0])

# features.loc[features['color_parsed2'].notnull(), 'color_parsed2'] = features.loc[features['color_parsed2'].notnull(), 'color_parsed2'].apply(lambda x: list(x)[0])

In [277]:
# группировка для определения топ цвета в разрезе категория-2,категория-3 для 1го товара
top_colors_by_cat_1 = features[features['color_parsed1'].notnull()] \
                        .groupby(['cat3_1', 'color_parsed1'], as_index=False).agg({'target':np.size})
# top_colors_by_cat_1

In [278]:
# группировка для определения топ цвета в разрезе категория-2,категория-3 для 2го товара
top_colors_by_cat_2 = features[features['color_parsed2'].notnull()] \
                        .groupby(['cat3_2', 'color_parsed2'], as_index=False).agg({'target':np.size})
# top_colors_by_cat_2

In [279]:
# features.iloc[features[(features['color_parsed1'].isnull()) | (features['color_parsed2'].isnull())].index,:]

In [280]:
# testdf = pd.concat([
#     features[features['color_parsed1'].isnull()].head(10),
#     features[features['color_parsed2'].isnull()].head(10)
# ], axis= 0).reset_index(drop=True)

In [285]:
for i in tqdm_notebook(range(len(features))):
    if features.loc[i,'color_parsed1'] is None:
        try:
            features.loc[i,'color_parsed1'] = top_colors_by_cat_1[(top_colors_by_cat_1['cat3_1'] == features.loc[i, 'cat3_1'])] \
                                                            .nlargest(1, 'target')['color_parsed1'].values[0]
        except:
            features.loc[i,'color_parsed1'] = 'черный'
    if features.loc[i,'color_parsed2'] is None:
        try:
            features.loc[i,'color_parsed2'] = top_colors_by_cat_2[(top_colors_by_cat_2['cat3_2'] == features.loc[i, 'cat3_2'])] \
                                                            .nlargest(1, 'target')['color_parsed2'].values[0]
        except:
            features.loc[i,'color_parsed2'] = 'черный'
    

  0%|          | 0/413790 [00:00<?, ?it/s]

In [286]:
features[['color_parsed1']].value_counts()

color_parsed1  
черный             234238
белый               30357
black               24140
серый               23439
серебристый         11315
                    ...  
гранитный               1
темно-оливковый         1
перу                    1
томатный                1
amber                   1
Length: 189, dtype: int64

In [287]:
features[['color_parsed2']].value_counts()

color_parsed2
черный           233057
белый             30682
black             24870
серый             23040
серебристый       10855
                  ...  
vanilla               1
pear                  1
терракота             1
томатный              1
amber                 1
Length: 193, dtype: int64

In [289]:
features[['color_parsed1','color_parsed2']].isnull().sum()

color_parsed1    0
color_parsed2    0
dtype: int64